Import datasets

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.stats import uniform
from scipy.stats import norm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import plot_confusion_matrix
import csv
import pandas as pd
import time
import os
from datetime import datetime, timedelta
import seaborn as sn
import requests
import json
%pip install haversine
import haversine as hs

READ DATA

In [2]:
#Read Data from February
header_CC=["codigoSerial", "fecha", "hora", "fechaHora", "temperatura", "humedad_relativa", "pm1_df", "pm10_df", "pm25_df", "pm1_nova", "pm10_nova", "pm25_nova", "calidad_temperatura", "calidad_humedad_relativa", "calidad_pm1_df", "calidad_pm10_df", "calidad_pm25_df", "calidad_pm1_nova", "calidad_pm10_nova", "calidad_pm25_nova"]
datatypes_CC={"codigoSerial":np.uint16, "temperatura":np.float16, "humedad_relativa":np.float16, "pm1_df":np.float32, "pm10_df":np.float32, "pm25_df":np.float32, "pm1_nova":np.float32, "pm10_nova":np.float32, "pm25_nova":np.float32}
df_CC = pd.read_csv("C:/Users/julio/Documents/UDEA/Maestría/DQ in IOT/Datasets/SIATA_CS/SplitDatosCC/Samples/"+"February.csv", header=None, names=header_CC, usecols=header_CC , dtype=datatypes_CC,parse_dates=["fecha","hora","fechaHora"])

#Data includes January, February and March
header_SS=["Fecha_Hora","codigoSerial","pm25","calidad_pm25","pm10","calidad_pm10"]
datatypes_SS={"codigoSerial":np.uint16,"pm25":np.float32,"pm10":np.float32}
df_SS = pd.read_csv("C:/Users/julio/Documents/UDEA/Maestría/DQ in IOT/Datasets/SIATA Stations/PM/"+"SS_PM.csv", header=None,names=header_SS, usecols=header_SS , dtype=datatypes_SS,parse_dates=["Fecha_Hora"])


grouped=df_CC.groupby(df_CC.codigoSerial)
CC={}
print("Citizen Scientist: ", sorted(list(df_CC.codigoSerial.unique())))
for i in df_CC.codigoSerial.unique():
    CC[i] = grouped.get_group(i).sort_values(by=['fechaHora'],ignore_index=True)
del df_CC

grouped=df_SS.groupby(df_SS.codigoSerial)
SS={}
print("Siata Stations: ", list(df_SS.codigoSerial.unique()))
for j in df_SS.codigoSerial.unique():
    SS[j] = grouped.get_group(j).sort_values(by=['Fecha_Hora'],ignore_index=True)
del df_SS
del grouped

Citizen Scientist:  [1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 99, 100, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 149, 150, 151, 152, 153, 154, 156, 157, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 213, 214, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 240, 241, 242, 243

READ DISTANCES

In [3]:
datatypesDistances={"codigoSerial_CC":np.uint16,"codigoSerial_ES":np.uint16,"Distancia_a_ES":np.float16}
Distances = pd.read_csv("C:/Users/julio/Documents/UDEA/Maestría/DQ in IOT/Datasets/Distances and positions/Distancias_2.csv", header=0, dtype=datatypesDistances,index_col="codigoSerial_CC")


DQ CALCULATIONS

In [ ]:
inicio="2020-02-01 00:00:00"
fin=   "2020-02-29 23:59:00"
nube=0
contador=0

acc_vs_dis=[]
missing_data_df=[]
missing_data_nova=[]

df_accur = pd.DataFrame(columns =['codigoSerial', 'dist', 'acc_nova', 'acc_df'])
df_comple = pd.DataFrame(columns =["codigoSerial","completeness_df","completeness_nova","completeness_group_df","completeness_group_nova"])
df_preci = pd.DataFrame(columns =["codigoSerial","precision_df","precision_nova","precision_group_df","precision_group_nova"])
df_uncer = pd.DataFrame(columns =["codigoSerial","uncertainty","uncertainty_group"])
df_conco = pd.DataFrame(columns =["codigoSerial","concordance_df_nova","concordance_df_siata","concordance_df_hum","concordance_df_temp","concordance_nova_siata","concordance_nova_hum","concordance_nova_temp"])

for nube in CC.keys():
    contador+=1
    CC[nube]["v_pm25"] = np.nan
    CC[nube]["alpha_df"] = np.nan
    CC[nube]["alpha_nova"] = np.nan
    #del df_window
    df_window=CC[nube][(CC[nube]['fechaHora'] >= inicio) & (CC[nube]['fechaHora'] <= fin)]
    
    #Remove outliers that are out of range, from documentation both nova and df range of measurements are [0,999]

    df_window=df_window.copy()
    df_window.loc[df_window["pm25_nova"]>999,"pm25_nova"]=np.nan
    df_window.loc[df_window["pm25_nova"]<0,"pm25_nova"]=np.nan
    df_window.loc[df_window["pm25_df"]>999,"pm25_df"]=np.nan
    df_window.loc[df_window["pm25_df"]<0,"pm25_df"]=np.nan
    
    #Remove data above the whiskers of the boxplot
    Q1 = df_window['pm25_df'].quantile(0.25)
    Q3 = df_window['pm25_df'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 
    df_window.loc[df_window["pm25_df"]>=Q3 + 1.5 *IQR,"pm25_df"]=np.nan
    
    Q1 = df_window['pm25_nova'].quantile(0.25)
    Q3 = df_window['pm25_nova'].quantile(0.75)
    IQR = Q3 - Q1    #IQR is interquartile range. 
    df_window.loc[df_window["pm25_nova"]>=Q3 + 1.5 *IQR,"pm25_nova"]=np.nan
    
    ref_date_range = pd.date_range(inicio, fin, freq='1Min')
    ref_date_range = pd.DataFrame(ref_date_range,columns=["ref_fechaHora"])
    
    #Hourly mean
    df_window['pm25_nova_ave']=np.nan
    df_window['pm25_df_ave']=np.nan
    #Hourly standar deviation
    df_window['pm25_nova_std']=np.nan
    df_window['pm25_df_std']=np.nan
    #Hourly uncertainty
    df_window['pm25_unc']=np.nan
    
    for ts in df_window['fechaHora']:
        if ts==ts.ceil('60min'):
            window=df_window[(df_window['fechaHora'] >= ts.floor('60min')) & (df_window['fechaHora'] < (ts+timedelta(minutes = 1)).ceil('60min'))]
            
        else:
            window=df_window[(df_window['fechaHora'] >= ts.floor('60min')) & (df_window['fechaHora'] < ts.ceil('60min'))]
        #print("Timestamp: ",ts,", Floor:",ts.floor('60min'),", Ceil:",ts.ceil('60min'),window['pm25_nova'].mean())
        #print(window['pm25_nova'])
        df_window.loc[df_window["fechaHora"]==ts,'pm25_nova_ave']=window['pm25_nova'].mean()
        df_window.loc[df_window["fechaHora"]==ts,'pm25_df_ave']=window['pm25_df'].mean()
        
        df_window.loc[df_window["fechaHora"]==ts,'pm25_nova_std']=100*(window['pm25_nova'].std()/window['pm25_nova'].mean())
        df_window.loc[df_window["fechaHora"]==ts,'pm25_df_std']=100*(window['pm25_df'].std()/window['pm25_df'].mean())
        
        df_window.loc[df_window["fechaHora"]==ts,'pm25_unc']=\
        100*np.sqrt((window.pm25_df-window.pm25_nova).pow(2).mean()/2)/((window.pm25_df+window.pm25_nova).mean()/2)
        
    del window

    prec_df=df_window.pm25_df_std.mean()
    prec_nova=df_window.pm25_nova_std.mean()
    uncer_df=df_window.pm25_unc.mean()
    
        
    df_preci=df_preci.append({"codigoSerial":nube,"precision_df":prec_df,"precision_nova":prec_nova}, ignore_index = True)
    print("%d. Nube: %d, Overall relative (Precision) Standard Deviation."%(contador,nube))
    
    df_uncer=df_uncer.append({"codigoSerial":nube,"uncertainty":uncer_df}, ignore_index = True)
    print("%d. Nube: %d, Overall relative Uncertainty BS, "%(contador,nube))
    
    Closest_Station=Distances.codigoSerial_ES.loc[nube]    
    if Closest_Station in SS.keys():
        #Clean values out of range
        SS[Closest_Station].loc[SS[Closest_Station]["pm25"]<=0,"pm25"]=np.nan
        for time in df_window.fechaHora:
            
            idx=SS[Closest_Station].Fecha_Hora.searchsorted(time,side="right")
            #print(idx, SS[Closest_Station].Fecha_Hora.loc[idx], time)
            v=SS[Closest_Station].loc[idx,"pm25"]
            df_window.loc[df_window.fechaHora == time,"v_pm25"]=v
            vm=df_window.loc[(df_window.fechaHora == time),"pm25_df_ave"]
            #print(time," : ",vm.values[0],"________",SS[Closest_Station].Fecha_Hora.loc[idx]," : ",v)
            df_window.loc[df_window.fechaHora == time,"alpha_df"]=100*abs(vm-v)/v
            vm=df_window.loc[(df_window.fechaHora == time),"pm25_nova_ave"]
            df_window.loc[df_window.fechaHora == time,"alpha_nova"]=100*abs(vm-v)/v 
        
        df_accur=df_accur.append({'codigoSerial':nube, 'dist':Distances.loc[nube,"Distancia_a_ES"], 'acc_df':df_window.alpha_df.mean(), 'acc_nova':df_window.alpha_nova.mean()}, ignore_index = True)
        print("%d. Nube: %d, Accuracy," %(contador, nube))
    
    ref_date_range = pd.date_range(inicio, fin, freq='1Min')
    ref_date_range = pd.DataFrame(ref_date_range,columns=["ref_fechaHora"])

    
    #Check for any missing date
    missing_dates = ref_date_range.loc[~ref_date_range.ref_fechaHora.isin(df_window.fechaHora),"ref_fechaHora"]

    #Add missing date rows
    for missing in missing_dates:
        df_window=df_window.append({"codigoSerial":nube,"fechaHora":missing}, ignore_index = True)
    
    #Check for any missing date
    missing_dates = ref_date_range.loc[~ref_date_range.ref_fechaHora.isin(df_window.fechaHora),"ref_fechaHora"]
    
    #Check for missing data
    missing_data_df=np.count_nonzero(np.isnan(df_window['pm25_df']))
    missing_data_nova=np.count_nonzero(np.isnan(df_window['pm25_nova']))
    comp_df=100*(1-missing_data_df/np.size(df_window.pm25_df))
    comp_nova=100*(1-missing_data_nova/np.size(df_window.pm25_nova))
    
    
    if comp_df<75:
        group_df=0
    elif 75<=comp_df:
        group_df=1
    
    if comp_nova<75:
        group_nova=0
    elif  75<=comp_nova:
        group_nova=1     
        
    df_comple=df_comple.append({"codigoSerial":nube,"completeness_df":comp_df,"completeness_nova":comp_nova,"completeness_group_df":group_df,"completeness_group_nova":group_nova}, ignore_index = True)
    
    print("%d. Nube: %d, Completeness." %(contador,nube))
    
    corr_df = df_window.loc[:,["pm25_df","pm25_nova","v_pm25","temperatura","humedad_relativa"]].corr().iloc[0].abs()
    corr_nova = df_window.loc[:,["pm25_df","pm25_nova","v_pm25","temperatura","humedad_relativa"]].corr().iloc[1].abs()
  
    
        
    df_conco=df_conco.append({"codigoSerial":nube,"concordance_df_nova":corr_df.pm25_nova,
                              "concordance_df_siata":corr_df.v_pm25,"concordance_df_hum":corr_df.humedad_relativa,"concordance_df_temp":corr_df.temperatura,
                              "concordance_nova_siata":corr_nova.v_pm25,"concordance_nova_hum":corr_nova.humedad_relativa,"concordance_nova_temp":corr_nova.temperatura}, 
                             ignore_index = True)
    
    print("%d. Nube: %d, Overall concordance, "%(contador,nube))

1. Nube: 220, Overall relative (Precision) Standard Deviation.
1. Nube: 220, Overall relative Uncertainty BS, 
1. Nube: 220, Accuracy,
1. Nube: 220, Completeness.
1. Nube: 220, Overall concordance, 
2. Nube: 1, Overall relative (Precision) Standard Deviation.
2. Nube: 1, Overall relative Uncertainty BS, 
2. Nube: 1, Accuracy,
2. Nube: 1, Completeness.
2. Nube: 1, Overall concordance, 
3. Nube: 192, Overall relative (Precision) Standard Deviation.
3. Nube: 192, Overall relative Uncertainty BS, 
3. Nube: 192, Accuracy,
3. Nube: 192, Completeness.
3. Nube: 192, Overall concordance, 
4. Nube: 261, Overall relative (Precision) Standard Deviation.
4. Nube: 261, Overall relative Uncertainty BS, 
4. Nube: 261, Accuracy,
4. Nube: 261, Completeness.
4. Nube: 261, Overall concordance, 
5. Nube: 262, Overall relative (Precision) Standard Deviation.
5. Nube: 262, Overall relative Uncertainty BS, 
5. Nube: 262, Accuracy,
5. Nube: 262, Completeness.
5. Nube: 262, Overall concordance, 
6. Nube: 104, O

In [5]:
import pandas as pd
import time

inicio="2020-02-01 00:00:00"
fin=   "2020-02-29 23:59:00"
nube=0
contador=0
    
for nube in [220]:
    ref_date_range = pd.date_range(inicio, fin, freq='1Min')
    ref_date_range = pd.DataFrame(ref_date_range,columns=["ref_fechaHora"])

    
    #Check for any missing date
    missing_dates = ref_date_range.loc[~ref_date_range.ref_fechaHora.isin(df_window.fechaHora),"ref_fechaHora"]

    #Add missing date rows
    for missing in missing_dates:
        df_window=df_window.append({"codigoSerial":nube,"fechaHora":missing}, ignore_index = True)
    
    #Check for any missing date
    missing_dates = ref_date_range.loc[~ref_date_range.ref_fechaHora.isin(df_window.fechaHora),"ref_fechaHora"]
    
    #Check for missing data
    missing_data_df=np.count_nonzero(np.isnan(df_window['pm25_df']))
    missing_data_nova=np.count_nonzero(np.isnan(df_window['pm25_nova']))
    comp_df=100*(1-missing_data_df/np.size(df_window.pm25_df))
    comp_nova=100*(1-missing_data_nova/np.size(df_window.pm25_nova))
    
    
    if comp_df<75:
        group_df=0
    elif 75<=comp_df:
        group_df=1
    
    if comp_nova<75:
        group_nova=0
    elif  75<=comp_nova:
        group_nova=1  

NameError: name 'df_window' is not defined

In [6]:
ref_date_range

,ref_fechaHora
0,2020-02-01 00:00:00
1,2020-02-01 00:02:00
2,2020-02-01 00:04:00
3,2020-02-01 00:06:00
4,2020-02-01 00:08:00
...,...
20875,2020-02-29 23:50:00
20876,2020-02-29 23:52:00
20877,2020-02-29 23:54:00
20878,2020-02-29 23:56:00
